# Circuit Trace Demo
Use `rg-tracer trace` to materialise circuit traces for a run.

## Attribution Delta Panel
Use attribution metrics to contextualise circuit traces across training phases.

In [ ]:
from pathlib import Path
import json
from pprint import pprint

NO_ATTR_RUNS_MSG = 'No runs with attribution metrics found; execute a self-play run first.'

run_dir = Path('runs').resolve()
if not run_dir.exists() or not run_dir.is_dir():
    raise RuntimeError(NO_ATTR_RUNS_MSG)
existing = [
    path
    for path in run_dir.iterdir()
    if path.is_dir() and (path / 'attr_metrics.jsonl').exists()
]
if not existing:
    raise RuntimeError(NO_ATTR_RUNS_MSG)
latest = max(existing, key=lambda path: path.stat().st_mtime)
metrics_path = latest / 'attr_metrics.jsonl'
records = []
for line in metrics_path.read_text(encoding='utf-8').splitlines():
    if not line.strip():
        continue
    try:
        records.append(json.loads(line))
    except json.JSONDecodeError:
        continue

def _alignment_key(item):
    value = item.get('delta_alignment') if isinstance(item, dict) else 0.0
    try:
        return float(value)
    except (TypeError, ValueError):
        return 0.0

top_3 = sorted(records, key=_alignment_key, reverse=True)[:3]
pprint(top_3)
